In [2]:
import pandas as pd

# Baca file Excel yang memiliki beberapa sheet
input_excel_file = 'data_resampling.xlsx'
xls = pd.ExcelFile(input_excel_file)

# Siapkan dictionary untuk menampung dataframe hasil interpolasi
daily_data = {}

# Loop melalui setiap sheet
for sheet_name in xls.sheet_names:
    # Baca data dari sheet
    df = pd.read_excel(input_excel_file, sheet_name=sheet_name)
    
    # Menghapus kolom yang tidak diperlukan
    df = df.drop(columns=['CORR_GAS_RES_RATE_MMSCFD', 'CORR_WTR_RATE_STBD', 'Universal'])
    
    # Konversi kolom DATE_STAMP menjadi datetime
    df['DATE_STAMP'] = pd.to_datetime(df['DATE_STAMP'], format='%d/%m/%Y')
    
    # Mengatasi tanggal duplikat dengan melakukan agregasi rata-rata
    df = df.groupby('DATE_STAMP').mean().reset_index()
    
    # Set DATE_STAMP sebagai index
    df.set_index('DATE_STAMP', inplace=True)
    
    # Resample menjadi frekuensi harian dan lakukan interpolasi
    df_daily = df.resample('D').interpolate()
    
    # Reset index untuk membuat DATE_STAMP menjadi kolom kembali
    df_daily.reset_index(inplace=True)
    
    # Simpan hasilnya di dictionary dengan nama sheet sebagai kunci
    daily_data[sheet_name] = df_daily

# Tentukan nama file output Excel
output_excel_file = 'data_resampling_daily.xlsx'

# Tulis hasilnya ke file Excel baru
with pd.ExcelWriter(output_excel_file) as writer:
    for sheet_name, df_daily in daily_data.items():
        df_daily.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Data harian dengan interpolasi telah disimpan ke dalam file {output_excel_file}")


Data harian dengan interpolasi telah disimpan ke dalam file data_resampling_daily.xlsx
